In [1]:
%env http_proxy=http://127.0.0.1:10890
%env https_proxy=http://127.0.0.1:10890
%env no_proxy=127.0.0.1,localhost
%env HTTP_PROXY=http://127.0.0.1:10890
%env HTTPS_PROXY=http://127.0.0.1:10890
%env NO_PROXY=127.0.0.1,localhost
!echo -e "\033[32m[√] 已开启代理\033[0m"
from dotenv import load_dotenv
load_dotenv()

env: http_proxy=http://127.0.0.1:10890
env: https_proxy=http://127.0.0.1:10890
env: no_proxy=127.0.0.1,localhost
env: HTTP_PROXY=http://127.0.0.1:10890
env: HTTPS_PROXY=http://127.0.0.1:10890
env: NO_PROXY=127.0.0.1,localhost
[√] 已开启代理


True

In [2]:
from typing import List

from assistant import check_local
from search import search_github

import os
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from search import search_github
from download import load_readme
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
embeddings = OpenAIEmbeddings()
def download_readme_to_db(keywords: List[str]) -> str:
    """
    Search github repos with a list of detailed keywords, and save the readme of them into a vector database. Return the path of db
    """
    keywords = [keyword.lower().strip() for keyword in keywords][:1]
    keywords = ["gaussian splatting"]
    db_path = check_local(keywords)
    # vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    if not db_path:
        repos = search_github(keywords, 1)
        vector_store = FAISS(
                        embedding_function=embeddings,
                        index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
                        docstore=InMemoryDocstore(),
                        index_to_docstore_id={},
                    )
        load_readme(repos, vector_store)
        db_file_name = '-'.join(keywords).lower()
        db_path = f'./db/{db_file_name}'
        vector_store.save_local(db_path)
    return db_path

def search_db(db_path: str, text: str) -> List[str]:
    """
    Search with the text from a vector db loaded from db_path, and return the descriptions and readme of top related repos
    """
    vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    bm25_retriever = BM25Retriever.from_documents(vector_store.docstore._dict.values())
    faiss_retriever = vector_store.as_retriever()
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever], weights=[0.8, 0.2]
    )
    similar_repos = ensemble_retriever.invoke(text)
    return [{"repo_name": repo.metadata["repo_name"], "repo_readme": repo.page_content} for repo in similar_repos]

In [3]:
from swarm import Swarm, Agent

client = Swarm()

def transfer_to_writer():
    return writer

def transfer_to_tool_user():
    return tool_user


def transfer_to_reviewer():
    return reviewer

tool_user = Agent(
    name="Tool_user A",
    instructions="""Tool_user. You need to plan how to use the tools you have and find the GitHub repository that best meets user's requirements.. 
    Capable of understanding a detailed algorithm description provided by the user, interpreting the algorithm code needed, and determining the requirements for the code (such as the programming language, whether it can run on CPU/GPU, etc.). 
    You will find the GitHub repository that best meets these requirements. 
    1. You should use download_readme_to_db to get the path of db before search the database.
    2. Use the text generated by Writer to seach the database.
    """,
    functions=[transfer_to_writer, transfer_to_reviewer, download_readme_to_db, search_db],
)

writer = Agent(
    name="Writer",
    instructions="""Writer. Your task is to generate a README file for a project that aligns as closely as possible with the user's specified algorithm requirements. 
    The README must highlight and prioritize the user's specific needs, such as programming language, hardware compatibility (CPU/GPU), performance optimization, and any other explicit details provided in the algorithm description.
    """,
    functions=[transfer_to_tool_user],
)

reviewer = Agent(
    name="Reviewer",
    instructions="""Reviewer. Review the top related repos provided by Tool_user after use search_db and select the one that best matches the user's algorithm requirements. 
    After returning the repository name, output TERMINATE to signal the completion of the task.
    """
)

response = client.run(
    agent=tool_user,
    messages=[{"role": "user", "content": "Give me the most related github repo whose code is written with C kernel and can be run on the CPU of this title and abstract: 3D Gaussian Splatting for Real-Time Radiance Field Rendering Radiance Field methods have recently revolutionized novel-view synthesis of scenes captured with multiple photos or videos. However, achieving high visual quality still requires neural networks that are costly to train and render, while recent faster methods inevitably trade off speed for quality. For unbounded and complete scenes (rather than isolated objects) and 1080p resolution rendering, no current method can achieve real-time display rates. We introduce three key elements that allow us to achieve state-of-the-art visual quality while maintaining competitive training times and importantly allow high-quality real-time (≥ 30 fps) novel-view synthesis at 1080p resolution. First, starting from sparse points produced during camera calibration, we represent the scene with 3D Gaussians that preserve desirable properties of continuous volumetric radiance fields for scene optimization while avoiding unnecessary computation in empty space; Second, we perform interleaved optimization/density control of the 3D Gaussians, notably optimizing anisotropic covariance to achieve an accurate representation of the scene; Third, we develop a fast visibility-aware rendering algorithm that supports anisotropic splatting and both accelerates training and allows realtime rendering. We demonstrate state-of-the-art visual quality and real-time rendering on several established datasets."}],
)

print(response.messages[-1]["content"])

Based on your requirements, here's a GitHub repository related to "3D Gaussian Splatting for Real-Time Radiance Field Rendering" that seems to match your criteria, although it's worth noting that it primarily uses PyTorch and OpenGL, which involves GPU usage. 

**Repository Name:** [ue-abyar/Persian-Gaussian-Splatting](https://github.com/ue-abyar/Persian-Gaussian-Splatting)

The repository contains:

- An OpenGL-based real-time viewer to render trained models.
- A PyTorch-based optimizer for 3D Gaussian model production.
- Dependency on a CUDA-ready GPU for the real-time viewer.

While the optimizer utilizes GPU (PyTorch), the real-time viewer is OpenGL-based. If you want something completely C/C++ kernel-based that's strictly CPU-only, you might need to adjust the current code or search more specifically due to the GPU dependencies described herein. Adjustments can involve removing CUDA elements if a CPU-only implementation is essential, although this might heavily affect performance.